In [ ]:
import pandas

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [ ]:
db_twitter = client["Twitter"]

In [ ]:
print("twitter-australia: {:,}".format(db_twitter["twitter-australia"].find({}).count()))
print("twitter-other: {:,}".format(db_twitter["twitter-other"].find({}).count()))

# Twitter
##  Compute the number of tweets based on the user location

In [ ]:
# Construct pipline for aggregation

pipeline = [
    { "$group": {
        "_id": "$user.location",
        "count": { "$sum": 1 }
        }
    }
]

In [ ]:
#  Twitter-Australia data

aus_data = list(db_twitter['twitter-australia'].aggregate(pipeline))
print(aus_data)

In [ ]:
# Print total number of tweets in Australia

total = 0
print('Tweet counts - Australia:\n')
for data in aus_data:
    print(data['_id']+':', data['count'])
    total += data['count']
        
print('\nTotal:', total)

In [ ]:
# get list of all city name of australia

colnames = ['city', 'state', 'country']
data = pandas.read_csv('au.csv', names=colnames)

In [ ]:
ci = data.city.tolist()
city = ci[1:]
st = data.state.tolist()
state = st[1:]
co = data.country.tolist()
country = co[1:]

In [ ]:
aus_data_format = []

for data in aus_data:
    exist = 0
    
    for x in city:  
        loc = data["_id"].lower().replace("australia","")
        if x.lower() in loc:
            index = city.index(x)
            if len(aus_data_format)>0:
                for d in aus_data_format:
                    if x in d["id"]:
                        d["count"] += data["count"] 
                        exist = 1
                        break 
                if exist == 0:
                    aus_data_format.append({"id":x + ',' + state[index] + "," + country[index],"count":data["count"]})
                    exist = 1
            else: 
                aus_data_format.append({"id":x + ',' + state[index] + "," + country[index],"count":data["count"]})
                exist = 1
            break
    if exist == 0:
        for z in state:
            if z.lower() in data["_id"].lower():
                index = state.index(z)
                if len(aus_data_format)>0:
                    for d in aus_data_format:
                        s = d["id"].split(",")[0]
                        if (s == "null") and (z in d["id"]):
                            d["count"] += data["count"]  
                            exist = 1
                            break 
                    if exist == 0:
                        aus_data_format.append({"id":"null" + "," + z + "," + country[index],"count":data["count"]})
                        exist = 1
                else: 
                    aus_data_format.append({"id": "null" + ","  + z + "," + country[index],"count":data["count"]})  
                    exist = 1
                break
    if exist == 0:
        for y in country:
            if y.lower() in data["_id"].lower():
                if len(aus_data_format)>0:
                    for d in aus_data_format:
                        s = d["id"].split(",")[1]
                        if (s == "null") and (y in d["id"]):
                            d["count"] += data["count"]       
                            exist = 1
                            break 
                    if exist == 0:
                        aus_data_format.append({"id":"null" + "," + "null" + ',' + y,"count":data["count"]})
                        exist = 1
                else: 
                    aus_data_format.append({"id": "null" + ","  + "null" + ',' + y,"count":data["count"]}) 
                    exist = 1
                #print(aus_data_format)
                break

In [ ]:
# Print total number of tweets in Australia
file_name = 'twitter-australia-user-collection.csv'

with open(file_name, 'w') as f:
    # header
    f.write('city,state,country,Tweets from Australia\n')
    
    for data in aus_data_format:
        city = data['id'].split(",")[0]
        state = data['id'].split(",")[1]
        country = data['id'].split(",")[2]
        if state == 'Victoria':
            state = 'VIC'
        
        line = city + ',' + state + ',' +  country + ',' + str(data['count']) + '\n'
        f.write(line)
    
print (file_name, 'is ready.')